In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset


class FeatureNameDataset(Dataset):
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path, sep='_!_', header=None, engine='python', names=['id','feature_id', 'feature_name', 'content'])
        self.data = self.data.drop_duplicates(subset=['feature_id'])
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        feature_id = self.data.iloc[idx, 0]
        feature_name = self.data.iloc[idx, 1]

        sample = {'feature_id': feature_id, 'feature_name': feature_name}

        if self.transform:
            sample = self.transform(sample)

        return sample

class FeatureContentDataset(Dataset):
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path, sep='_!_', header=None, engine='python', names=['id','feature_id', 'feature_name', 'content'])
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        feature_id = self.data.iloc[idx, 0]
        content = self.data.iloc[idx, 2]

        sample = {'feature_id': feature_id, 'content': content}

        if self.transform:
            sample = self.transform(sample)

        return sample

# 示例使用示范
# 用"_!_"作为分隔符读取CSV文件
file_path = '/home/liyi/gpt/data/toutiao_cat_data.txt'  # 请替换为您的文件路径
feature_name_dataset = FeatureNameDataset(file_path)

# 创建特征和内容对应关系的数据集
feature_content_dataset = FeatureContentDataset(file_path)

# 获取特征名称对应关系数据集的大小
print("Feature Name Dataset size:", len(feature_name_dataset))

# 获取特征和内容对应关系数据集的大小
print("Feature Content Dataset size:", len(feature_content_dataset))

# 获取第一个特征名称对应关系数据集的样本
first_feature_name_sample = feature_name_dataset[0]
print("First Feature Name Sample:", first_feature_name_sample)

# 获取第一个特征和内容对应关系数据集的样本
first_feature_content_sample = feature_content_dataset[0]
print("First Feature Content Sample:", first_feature_content_sample)

Feature Name Dataset size: 15
Feature Content Dataset size: 382688
First Feature Name Sample: {'feature_id': 101, 'feature_name': 'news_culture'}
First Feature Content Sample: {'feature_id': 101, 'content': '京城最值得你来场文化之旅的博物馆'}


In [4]:
from sklearn.model_selection import train_test_split

train_dataset, test_dataset = train_test_split(feature_content_dataset, test_size=0.2, random_state=42)

# 获取训练集和测试集的大小
print("Train Dataset size:", len(train_dataset))
print("Test Dataset size:", len(test_dataset))

Train Dataset size: 306150
Test Dataset size: 76538


In [5]:
import torchtext
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('京城最值得你来场文化之旅的博物馆')

['京城最值得你来场文化之旅的博物馆']

In [6]:
!pip install jieba

In [7]:
import jieba

def tokenize_chinese(text):
    return list(jieba.cut(text))

tokenize_chinese('京城最值得你来场文化之旅的博物馆')

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.414 seconds.
Prefix dict has been built successfully.


['京城', '最', '值得', '你', '来场', '文化', '之旅', '的', '博物馆']

In [8]:
import collections
import torchtext
counter = collections.Counter()


for item in train_dataset:
    line = item['content']
    counter.update(tokenize_chinese(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

In [9]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi.get(s,0) for s in tokenize_chinese(x)]

encode('我爱北京天安门')
stoi.get('好运来',0)


Vocab size if 153247


0

In [10]:
print(train_dataset[0])

{'feature_id': 116, 'content': '大学宿舍里最怕碰上这种人，简直无法容忍'}


In [11]:
vocab

Vocab()

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer=tokenize_chinese)
corpus = [
        '我爱吃热狗.',
        '这只狗跑得好快',
        '外面好热',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['我家的狗需要在夏天吃热狗']).toarray()

array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0]])

In [13]:
vectorizer.get_feature_names()

['.', '只', '吃', '外面', '好', '得', '快', '我', '热', '热狗', '爱', '狗', '跑', '这']

In [14]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0]['content']))

tensor([1., 1., 1.,  ..., 0., 0., 0.])


In [15]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t['feature_id']-100 for t in b]),
            torch.stack([to_bow(t['content']) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

In [16]:
len(train_dataset)

306150

In [17]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,17),torch.nn.LogSoftmax(dim=1))
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [18]:
train_epoch(net,train_loader,epoch_size=30000)

3200: acc=0.4209375
6400: acc=0.52390625
9600: acc=0.583125
12800: acc=0.620234375
16000: acc=0.646125
19200: acc=0.66671875
22400: acc=0.6829464285714286
25600: acc=0.692890625
28800: acc=0.7041319444444445


(0.0683801402923649, 0.7072228144989339)

In [34]:
loss_fn = torch.nn.NLLLoss()

def test_model(net, dataloader):
    net.eval()  # 设置模型为评估模式
    total_loss, acc, count = 0, 0, 0
    with torch.no_grad():  # 禁用梯度计算
        for labels, features in dataloader:
            out = net(features)
            loss = loss_fn(out, labels)
            total_loss += loss
            _, predicted = torch.max(out, 1)
            acc += (predicted == labels).sum()
            count += len(labels)

    avg_loss = total_loss.item() / count
    accuracy = acc.item() / count
    print(f"Test Loss: {avg_loss}, Test Accuracy: {accuracy}")
    return avg_loss, accuracy

test_loss, test_accuracy = test_model(net, test_loader)


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [3., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ...

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [2., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ...

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [2., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ...

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [2., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [2., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [2., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ...

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[2., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ...

...

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [2., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [2., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ...

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [3., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [2., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [2., 0., 0.,  ...

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [3., 0., 0.,  ...

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [3., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ...

KeyboardInterrupt: 

In [48]:

input_tensor = to_bow('没有爱，就没有教育！江门开平一幼儿园收到家长亲笔写的“情书”_!_美丽中国,幼儿园,牛津,江门,Nini,感谢信')
tensor_2d = input_tensor.unsqueeze(0)
def getFeatureName(feature_id):
    return next((item for item in feature_name_dataset if item['feature_id'] == feature_id), None)
        
with torch.no_grad():  # 禁用梯度计算
    out = net(tensor_2d)
    probabilities = torch.exp(out)
    predicted_class = torch.argmax(probabilities, dim=1)
    print(predicted_class)
    print(getFeatureName(100+predicted_class))



tensor([8])
{'feature_id': 108, 'feature_name': 'news_edu'}


In [49]:
def get_class(text):
    input_tensor = to_bow(text)
    tensor_2d = input_tensor.unsqueeze(0)
    with torch.no_grad():
        out = net(tensor_2d)
        probabilities = torch.exp(out)
        predicted_class = torch.argmax(probabilities, dim=1)
        print(predicted_class)
        print(getFeatureName(100+predicted_class))  
get_class('佟丽娅和陈思成离婚了')

tensor([2])
{'feature_id': 102, 'feature_name': 'news_entertainment'}


In [50]:
get_class('佟丽娅又结婚了')

tensor([2])
{'feature_id': 102, 'feature_name': 'news_entertainment'}


In [51]:
get_class('梅西又获得了金球奖')

tensor([3])
{'feature_id': 103, 'feature_name': 'news_sports'}


In [54]:
get_class('2024年的金球产生了')

tensor([10])
{'feature_id': 110, 'feature_name': 'news_military'}


In [56]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), tokenizer=tokenize_chinese, min_df=1)
corpus = [
        '我爱吃热狗.',
        '这只狗跑得好快',
        '外面好热',
    ]

bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['我家的狗需要在大热天吃热狗']).toarray()

Vocabulary:
 {'我': 13, '爱': 18, '吃': 3, '热狗': 16, '.': 0, '我 爱': 14, '爱 吃': 19, '吃 热狗': 4, '热狗 .': 17, '这': 24, '只': 1, '狗': 20, '跑': 22, '得': 10, '好': 7, '快': 12, '这 只': 25, '只 狗': 2, '狗 跑': 21, '跑 得': 23, '得 好': 11, '好 快': 8, '外面': 5, '热': 15, '外面 好': 6, '好 热': 9}


array([[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 0, 0, 0]])